In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split

2024-05-06 09:08:42.638299: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-06 09:08:42.638468: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-06 09:08:42.797556: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
train_df = pd.read_csv("/kaggle/input/smtitanic/train.csv")

# Dropping null values from the dataset and shuffling.
train_df.dropna(subset = ["Age"], inplace = True)
train_df = train_df.reindex(np.random.permutation(train_df.index))

# All our needed features
train_x_age = train_df["Age"]
train_x_gender = train_df["Sex"].map({ "male": 1, "female": 2})
train_x_class = train_df["Pclass"]

In [3]:
# Spliting the data
X = pd.DataFrame()
X["Class"] = train_x_class
X["Gender"] = train_x_gender
X["Age"] = train_x_age

Y = train_df["Survived"]

train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.2, random_state=42)

In [4]:
def build_model(lr):
    
    # This is were the model is declared and all the layers.
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(1,)),
        tf.keras.layers.Dense(6, activation="relu"),
        tf.keras.layers.Dense(1, activation="sigmoid")
    ])

    model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=lr),
                      loss="mean_squared_error",
                      metrics=[tf.keras.metrics.RootMeanSquaredError()])
    return model

def train_model(model, train_x, train_y, ep, bs, vs):
    model.fit(train_x, train_y, epochs = ep, batch_size = bs, validation_split = vs)

In [5]:
ep = 4
bs = 10
lr = 0.03
vs = 0.6

model = build_model(lr)
train_model(model, [train_x["Class"], train_x["Gender"], train_x["Age"]], train_y, ep, bs, vs)

Epoch 1/4
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.2306 - root_mean_squared_error: 0.4800 - val_loss: 0.2216 - val_root_mean_squared_error: 0.4707
Epoch 2/4
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2082 - root_mean_squared_error: 0.4562 - val_loss: 0.2160 - val_root_mean_squared_error: 0.4648
Epoch 3/4
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2127 - root_mean_squared_error: 0.4609 - val_loss: 0.2183 - val_root_mean_squared_error: 0.4672
Epoch 4/4
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2128 - root_mean_squared_error: 0.4611 - val_loss: 0.2185 - val_root_mean_squared_error: 0.4675


In [6]:
test_df_survived = model.predict([test_x["Class"], test_x["Gender"], test_x["Age"]])
predictions = np.round(test_df_survived).astype(int)

# We create a data frame to compare our predictions with the input
test_x["Gender"] = test_x["Gender"].map({ 1: "male", 2: "female"})
test_x["Survival Prediction"] = predictions

test_x.head(10)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 


,Class,Gender,Age,Survival Prediction
608,2,female,22.0,0
529,2,male,23.0,0
714,2,male,52.0,0
660,1,male,50.0,1
194,1,female,44.0,1
299,1,female,50.0,1
757,2,male,18.0,0
174,1,male,56.0,1
211,2,female,35.0,0
262,1,male,52.0,1


In [7]:
i = 0
cp = 0
for prediction in predictions:
    if prediction == test_y.iloc[i]:
        cp=cp+1
    i=i+1

print(cp/i)

0.6853146853146853


In [8]:
test_df = pd.read_csv("/kaggle/input/smtitanic/test.csv")

test_df_class = test_df["Pclass"].ffill()
test_df_gender = test_df["Sex"].map({ "male":1 , "female": 2})
test_df_age = test_df["Age"].ffill()

model_predictions =  model.predict([test_df_class, test_df_gender, test_df_age])
rounded_predictions = np.round(model_predictions).astype(int)

submisson_df = pd.DataFrame(test_df["PassengerId"])
submisson_df["Survived"] = rounded_predictions

submisson_df

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [9]:
submisson_df.to_csv("/kaggle/working/submission_1.csv")
print("")